# Download Searchanalytics Data
Herunterladen von Google Search Console Suchanalyse-Daten mit dem Python Package `searchconsole` [github link](https://github.com/joshcarty/google-searchconsole)
- Beispiel aus `google_searchanalytics.ipynb`
- Nur der Part für Download mit ipywidgets

In [ ]:
from googleapiclient.discovery import build
from searchconsole.account import Account
from tqdm import tqdm_notebook as tqdm
from pydata_google_auth import cache
from IPython.display import FileLink
import ipywidgets as widgets
import pydata_google_auth
import searchconsole
import pandas as pd
import time
import json

with open('client_secrets.json') as f:
    cs = json.load(f)

credentials = pydata_google_auth.get_user_credentials(scopes=['https://www.googleapis.com/auth/webmasters.readonly'],
                                                      client_id=cs['installed']['client_id'],
                                                      client_secret=cs['installed']['client_secret'],
                                                      credentials_cache=cache.REAUTH,
                                                      auth_local_webserver=False)

service = build('webmasters', 'v3', credentials=credentials)

account = Account(service, credentials)

In [ ]:
def download_data(b):
    output.clear_output()
    with output:
        assert start.value is not None, 'Startdate not set'
        assert end.value is not None, 'Enddate not set'
        assert start.value <= end.value, 'Startdate after Enddate'

        export_to_csv_button.disabled = True
        b.description = '...in progress...'
        b.disabled = True
        try:
            global daily_data

            dates = [row.strftime('%Y-%m-%d') for row in pd.date_range(start=start.value, end=end.value, freq='D')]

            daily_data = []

            for date in tqdm(dates):
                data = webproperty_select.value.query.range(
                    start=date, stop=date
                ).dimension(
                    *dimension.value
                ).search_type(
                    searchtype.value
                ).get().to_dataframe()
                data['date'] = date
                daily_data.append(data)
                time.sleep(.3)

            daily_data = pd.concat(daily_data)
            daily_data['date'] = pd.to_datetime(daily_data['date'])
        except:
            raise
        else:
            export_to_csv_button.disabled = False
        finally:
            b.disabled = False
            b.description = 'Download from API'

def export_data(b):
    output.clear_output()
    with output:
        print('Exporting to CSV...')
        daily_data.to_csv(
            'gsc_searchanalytics_output.csv',
            header=True,
            index=False,
            sep=';',
            decimal=','
        )
        print('Done ✌️😉')
        display(FileLink('gsc_searchanalytics_output.csv'))

webproperty_select = widgets.Dropdown(
    options=[(row.url, row) for row in account.webproperties],
    description='Property',
    layout=widgets.Layout(flex='2 1 auto')
)

start = widgets.DatePicker(description='Startdate')
end = widgets.DatePicker(description='Enddate')

searchtype = widgets.Dropdown(
    options=sorted(['web','image','video']),
    value='web',
    description='Searchtype'
)

dimension = widgets.SelectMultiple(
    options=sorted(['country','device','page','query']),
    value=['query'],
    description='Dimensions'
)

download_data_button = widgets.Button(
    description='Download from API',
    tooltip='Start calling GSC API',
    layout=widgets.Layout(flex='2 1 auto')
)

export_to_csv_button = widgets.Button(
    description='Export to CSV',
    tooltip='Write GSC data to CSV file',
    disabled=True,
    layout=download_data_button.layout
)

download_data_button.on_click(download_data)
export_to_csv_button.on_click(export_data)

output = widgets.Output(
    layout=widgets.Layout(flex='2 1 auto')
)

widgets.VBox([
    widgets.HBox([webproperty_select,start,end]),
    widgets.HBox([dimension,searchtype]),
    widgets.HBox([download_data_button,
                  export_to_csv_button]),
    widgets.HBox([output])
])